# Using the Illumina InterOp Library in Python: Part 5

## Install 

If you do not have the Python InterOp library installed, then you can do the following:

    $ pip install interop

You can verify that InterOp is properly installed:

    $ python -m interop --test

## Before you begin
If you plan to use this tutorial in an interactive fashion, then you should get an example run folder that contains InterOp files such as TileMetricsOut.bin.

Please change the path below so that it points at the run folder you wish to use:

In [44]:
run_folder = r""

## Getting SAV Imaging Tab-like Metrics
The `run_metrics` class encapsulates the model for all the individual InterOp files as well as containing information
from the `RunInfo.xml`. The [Modules](https://git.illumina.com/pages/Bioinformatics/interop/modules.html) page contains a subset of the applications programmer's interface
for all the major classes in C++. The available Python models all have the same names (with a few exceptions) and take
the same parameters. This page is useful for accessing specific values loaded from the individual files.

In [45]:
from interop import py_interop_run_metrics, py_interop_run, py_interop_table
import numpy
import pandas as pd

In [46]:
run_metrics = py_interop_run_metrics.run_metrics()

By default, the `run_metrics` class loads all the InterOp files.

> run_folder = run_metrics.read(run_folder)

The InterOp library can provide a list of all necessary InterOp files for a specific application. The following shows how to generate that list for the index summary statistics:

In [47]:
valid_to_load = py_interop_run.uchar_vector(py_interop_run.MetricCount, 0)

In [48]:
py_interop_table.list_imaging_table_metrics_to_load(valid_to_load)

The `run_metrics` class can use this list to load only the required InterOp files as follows:

In [49]:
run_metrics.read(run_folder, valid_to_load)

The column headers for the imaing table can be created as follows:

In [50]:
columns = py_interop_table.imaging_column_vector()
py_interop_table.create_imaging_table_columns(run_metrics, columns)

Convert the columns object to a list of strings.

In [54]:
headers = []
for i in range(columns.size()):
    column = columns[i]
    if column.has_children():
        headers.extend([column.name()+"("+subname+")" for subname in column.subcolumns()])
    else:
        headers.append(column.name())

Subsample rows and columns

In [63]:
row_count=3
column_count=7

In [56]:
print headers[:column_count]

['Lane', 'Tile', 'Cycle', 'Read', 'Cycle Within Read', 'Density(k/mm2)', 'Density Pf(k/mm2)']


The data from imaging table can populate a numpy ndarray as follows:

In [41]:
column_count = py_interop_table.count_table_columns(columns)
row_offsets = py_interop_table.map_id_offset()
py_interop_table.count_table_rows(run_metrics, row_offsets)
data = numpy.zeros((row_offsets.size(), column_count), dtype=numpy.float32)
py_interop_table.populate_imaging_table_data(run_metrics, columns, row_offsets, data.ravel())

Convert the header list and data ndarray into a Pandas table.

In [64]:
d = []
for col, label in enumerate(headers[:column_count]):
    d.append( (label, pd.Series([val for val in data[:row_count, col]], index=[tuple(r) for r in  data[:row_count, :3]])))

Render the Imaging Table data using Pandas

In [65]:
df = pd.DataFrame.from_items(d)
print(df.to_string(index=False))

Lane    Tile  Cycle  Read  Cycle Within Read  Density(k/mm2)  Density Pf(k/mm2)
 1.0  1146.0    1.0   1.0                1.0     2961.699951        2338.800049
 1.0  1146.0    2.0   1.0                2.0     2961.699951        2338.800049
 1.0  1146.0    3.0   1.0                3.0     2961.699951        2338.800049
